In [112]:
import simpy
import random

class Local():
    
    def __init__(self, env):
        self.lst_paradas = []
        self.garagem_onibus = []
        self.env = env
        self.frota = None
        
    def add_parada(self, parada):
        self.lst_paradas.append(parada)
        parada.run()
        
    def add_onibus(self, onibus):
        self.garagem_onibus.append(onibus)
        #onibus.run()
        
    def add_frota(self, frota):
        self.frota = frota
    
    def escolher_destino(self, id):
        lst_temp = []
        for i in self.lst_paradas:
            if i.id > id:
                lst_temp.append(i.id)
        #return 3 
        return random.choice(lst_temp)
        
    def run(self, until):
        self.frota.run()
        self.env.run(until)
        
        
class Parada():
    
    def __init__(self, container, id, nome, tmcp, destino_final):
        self.env = container.env
        self.container = container
        self.id = id
        self.nome = nome
        self.tmcp = tmcp
        self.passageiros = []
        self.destino_final = destino_final
        
        self.container.add_parada(self)
    
    def run(self):
        if not self.destino_final:
            self.process = self.env.process(self.passageiro_generator())
    
    def passageiro_generator(self):
        ''' Processo que gerar passageiros para a parada de ônibuss'''
        cont = 0
        while True:
            try:
                destino = self.container.escolher_destino(self.id)
                
                p = Passageiro(self.env, cont, self.id, destino)
                
                yield self.env.timeout(random.expovariate(1.0/self.tmcp))
                
                p.chegada = env.now
                print("%.2f |%s informa: | Chegada do passageiro: %d com destino à parada %s" % (p.chegada, self.nome, p.id, p.destino))
                
                self.passageiros.append(p)
                cont += 1
            except simpy.Interrupt:
                pass

class Frota():
    
    def __init__(self, env, container, qtde_onibus, intervalo_saida):
        self.env = env
        self.container = container
        self.qtde_onibus = qtde_onibus
        self.intervalo_saida = intervalo_saida
        self.onibus_actual = -1
        for num in range(qtde_onibus):
            o = Onibus(self.container, 100) 
            self.container.add_onibus(o)
            
        self.container.add_frota(self)
    
    def run(self):
        self.onibus_actual += 1
        if self.onibus_actual < self.qtde_onibus:
            self.container.garagem_onibus[self.onibus_actual].run()
        
            
            
    
class Onibus():
    
    def __init__(self, container, capacidade):
        self.resource = simpy.Resource(container.env, capacity=1)
        self.container = container
        self.env = container.env
        self.capacidade = capacidade
        self.lst_passageiros = []
        self.tmep = 0.2 # tempo médio de embarque de passageiros
        
    
    def run(self):
        self.process = self.env.process(self.onibus_generator())
        
    
    def onibus_generator(self):
        ''' Processo que gerar onibus para percorrer pelas paradas'''
        
        for parada in self.container.lst_paradas:
            yield self.env.timeout(random.expovariate(8))
            print("%.2f | 🚌 (%d) - Saindo com destino à parada: %d " % (self.env.now, self.container.frota.onibus_actual, parada.id))
            yield self.env.timeout(4)
            print("%.2f | 🚍 (%d) - Chegou na parada %d " % (self.env.now, self.container.frota.onibus_actual, parada.id))
            numPassageiros = len(parada.passageiros)
            print("\t (%d) Número de passageiros na parada: %d" % (self.container.frota.onibus_actual, numPassageiros))
            print("\t (%d) Número de passageiros no ônibus: %d" % (self.container.frota.onibus_actual, len(self.lst_passageiros)))
            
            if len(self.lst_passageiros) > 0:
                yield self.env.process(self.desembarcar(self.lst_passageiros, parada.id))
                
            if numPassageiros > 0:
                yield self.env.process(self.embarcar(parada.passageiros))
            
        self.container.frota.run()
    
    def embarcar(self, lst):
        cont = 0
        print("\t (%d) #### Iniciando embarque ###" % (self.container.frota.onibus_actual))

        for p in lst:
            if cont < self.capacidade or cont < len(lst):
                request = self.resource.request()
                yield request
                yield self.env.timeout(random.expovariate(1.0/self.tmep))
                p.embarque = self.env.now
                print("\t\t   %.2f |(%d)| Embarque do passageiro %d" % (self.env.now, self.container.frota.onibus_actual, p.id))
                yield self.resource.release(request)
                self.lst_passageiros.append(p)
            else:
                if cont >= self.capacidade:
                    print("🚧 (%d) ônibus cheio, interrompendo o embarque!!" % self.container.frota.onibus_actual)
                    print("\t Restaram %d passageiros na parada de ônibus" % len(lst))
                else:
                    print("🚧 Todos os passageiros embarcaram")
                
                print("\t (%d) ônibus saindo..... para outra estação" % self.container.frota.onibus_actual)
        
        for p in self.lst_passageiros:
            lst.remove(p)
            
            
    
    def desembarcar(self, lst, id_parada):
        cont = 0
        temp =[]
        print("\t #### Iniciando Desembarque ###")
        for index, p in enumerate(lst):
            if p.destino == id_parada:
                yield self.env.timeout(random.expovariate(1.0/self.tmep))
                print("\t\t   %.2f | O passageiro %d, oriundo de parada %d desembarcou na parada %d" % (self.env.now, p.id, p.origem, p.destino))
                temp.append(p)
                self.container.lst_paradas[id_parada-1].passageiros.append(p)
                
        for i in temp:
            lst.remove(i)
                
            

class Passageiro():
    
    def __init__(self, env, id, origem, destino):
        self.env = env
        self.id = id
        self.origem = origem
        self.destino = destino           # id da parada de destino do passageiro
        self.chegada = None
        self.embarque = None
        self.chegada = None

        
env = simpy.Environment()
seed = random.seed(4)
ufpa_belem = Local(env)

# Criação de Paradas de ônibus (busStop)
p1 = Parada(ufpa_belem, 1, "Parada 1", 3, False) 
p2 = Parada(ufpa_belem, 2, "Parada 2", 7, False)
p3 = Parada(ufpa_belem, 3, "Parada 3 (Destino Final)", 0.0, True ) 

# Criação de ônibus para o campus (bus)
frota = Frota(env, ufpa_belem, 3, 10)

ufpa_belem.run(120)
# Criação de Passageiros 
        
        

        


0.01 | 🚌 (0) - Saindo com destino à parada: 1 
1.08 |Parada 1 informa: | Chegada do passageiro: 0 com destino à parada 2
2.62 |Parada 1 informa: | Chegada do passageiro: 1 com destino à parada 2
4.01 | 🚍 (0) - Chegou na parada 1 
	 (0) Número de passageiros na parada: 2
	 (0) Número de passageiros no ônibus: 0
	 (0) #### Iniciando embarque ###
		   4.30 |(0)| Embarque do passageiro 0
		   4.35 |(0)| Embarque do passageiro 1
4.45 | 🚌 (0) - Saindo com destino à parada: 2 
4.56 |Parada 2 informa: | Chegada do passageiro: 0 com destino à parada 3
7.46 |Parada 1 informa: | Chegada do passageiro: 2 com destino à parada 3
8.37 |Parada 1 informa: | Chegada do passageiro: 3 com destino à parada 2
8.45 | 🚍 (0) - Chegou na parada 2 
	 (0) Número de passageiros na parada: 1
	 (0) Número de passageiros no ônibus: 2
	 #### Iniciando Desembarque ###
		   8.78 | O passageiro 0, oriundo de parada 1 desembarcou na parada 2
		   9.10 | O passageiro 1, oriundo de parada 1 desembarcou na parada 2
	 (0) ###

ValueError: list.remove(x): x not in list